### The University of Melbourne, School of Computing and Information Systems
# COMP90086 Computer Vision, 2022 Semester 2

## Assignment

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

Question 1

In [ ]:
def ssd(imgL,imgR,window_size,max_offset):
    width, height = imgL.shape[:2]
    disparity = np.zeros((width, height), np.uint8)

    for i in tqdm(range(window_size//2, width-window_size//2)):
        for j in range(window_size//2, height-window_size//2):
            min_ssd = 65535
            for k in range(max_offset):
                if j-k < window_size//2:
                    break
                ssd = np.sum((imgL[i-window_size//2:i+window_size//2+1, j-window_size//2:j+window_size//2+1] - imgR[i-window_size//2:i+window_size//2+1, j-window_size//2-k:j+window_size//2+1-k])**2)
                if ssd < min_ssd:
                    min_ssd = ssd
                    disparity[i,j] = k
    return disparity

In [ ]:
def ncc(imgL,imgR,window_size,max_offset):
    width, height = imgL.shape[:2]
    disparity = np.zeros((width, height), np.uint8)

    for i in tqdm(range(window_size//2, width-window_size//2)):
        for j in range(window_size//2, height-window_size//2):
            max_ncc = -1
            for k in range(max_offset):
                if j-k < window_size//2:
                    break
                ncc = np.sum((imgL[i-window_size//2:i+window_size//2+1, j-window_size//2:j+window_size//2+1] - np.mean(imgL[i-window_size//2:i+window_size//2+1, j-window_size//2:j+window_size//2+1])) * (imgR[i-window_size//2:i+window_size//2+1, j-window_size//2-k:j+window_size//2+1-k] - np.mean(imgR[i-window_size//2:i+window_size//2+1, j-window_size//2-k:j+window_size//2+1-k]))) / (np.sqrt(np.sum((imgL[i-window_size//2:i+window_size//2+1, j-window_size//2:j+window_size//2+1] - np.mean(imgL[i-window_size//2:i+window_size//2+1, j-window_size//2:j+window_size//2+1]))**2)) * np.sqrt(np.sum((imgR[i-window_size//2:i+window_size//2+1, j-window_size//2-k:j+window_size//2+1-k] - np.mean(imgR[i-window_size//2:i+window_size//2+1, j-window_size//2-k:j+window_size//2+1-k]))**2)))
                if ncc > max_ncc:
                    max_ncc = ncc
                    disparity[i,j] = k
    return disparity

In [ ]:
# read images
imgL = cv2.imread('left.jpg', cv2.IMREAD_GRAYSCALE)
imgR = cv2.imread('right.jpg', cv2.IMREAD_GRAYSCALE)
# SSD
print("SSD")
disparity = ssd(imgL,imgR,5,64)
cv2.imwrite('ssd.png', disparity)
# NCC
print("NCC")
disparity = ncc(imgL,imgR,5,64)
cv2.imwrite('ncc.png', disparity)